In [2]:
import os
import requests
def get_vault_configuration(endpoint):
    vault_addr = os.environ.get("VAULT_ADDR")
    vault_token = os.environ.get("VAULT_TOKEN")
    endpoint = f"{vault_addr}/v1/kv/data/{endpoint}"

    # HTTP GET 요청을 통해 데이터를 가져옵니다.
    headers = {"X-Vault-Token": vault_token}
    response = requests.get(endpoint, headers=headers)

    if response.status_code == 200:
        data = response.json()
        return data['data']['data']

    else:
        # 에러 응답의 경우 예외를 발생시킵니다.
        response.raise_for_status()

ssh_info = get_vault_configuration("ssh")
ssh_ip = ssh_info['ssh_ip']['odroid']
id = ssh_info['ssh_id']
passwd = ssh_info['ssh_pass']

In [6]:
#mongodb 관련  database check
import pymongo

# MongoDB 서버 연결
client = pymongo.MongoClient(f'mongodb://{id}:{passwd}@{ssh_ip}:27017/')
db = client['FinanceData']
collection = db['marcap']
#print(db)
#print(collection)


In [3]:
# 데이터 확인
import pandas as pd
from datetime import datetime
import os
marcap_loc = "/Users/linetor/Documents/vscode/code_folder/crawler/stock/marcap/data/"
files = os.listdir(marcap_loc)

yearStr = datetime.today().strftime("%Y")

#파일명 출력하기
print("start")
from tqdm import tqdm
for fileName in tqdm(files):
    if yearStr not in fileName:
        df = pd.read_csv(marcap_loc+fileName)
        df['_id']=df['Code'].astype(str)+"_"+df['Date']
        df['filename']=fileName
        data_dict = df.to_dict('records')
        collection.insert_many(data_dict)
print("done")



start


 27%|██▋       | 8/30 [03:55<11:47, 32.14s/it]/var/folders/y5/11y4wdq91hv_96wtw7q0t8s40000gn/T/ipykernel_78859/2904538034.py:15: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(marcap_loc+fileName)
 43%|████▎     | 13/30 [05:15<04:52, 17.21s/it]/var/folders/y5/11y4wdq91hv_96wtw7q0t8s40000gn/T/ipykernel_78859/2904538034.py:15: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(marcap_loc+fileName)
 47%|████▋     | 14/30 [05:37<04:58, 18.65s/it]/var/folders/y5/11y4wdq91hv_96wtw7q0t8s40000gn/T/ipykernel_78859/2904538034.py:15: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(marcap_loc+fileName)
 83%|████████▎ | 25/30 [10:49<02:13, 26.73s/it]/var/folders/y5/11y4wdq91hv_96wtw7q0t8s40000gn/T/ipykernel_78859/2904538034.py:15: DtypeWarning: Columns (0) have mixed types. Specify dtype opt

done


In [ ]:
df['_id'].value_counts().sort_values()

In [ ]:
df = pd.read_csv(marcap_loc+"marcap-2021.csv.gz")
df['_id']=df['Code'].astype(str)+"_"+df['Date']
data_dict = df.to_dict('records')

In [ ]:
df['Code'].isna().sum()

In [ ]:
data_dict[0]

In [ ]:
df.head()
# Date : 날짜 (DatetimeIndex)
# Rank: 시가총액 순위 (당일)
# Code : 종목코드
# Name : 종명이름
# Open : 시가
# High : 고가
# Low : 저가
# Close : 종가
# Volume : 거래량
# Amount : 거래대금
# Changes : 전일대비
# ChangeCode: 등락 기호
# ChagesRatio : 전일대비 등락률
# Marcap : 시가총액(백만원)
# Stocks : 상장주식수
# MarketId : 시장기호
# Market : 시장
# Dept : 부서(한국거래소)

In [4]:
collection.insert_one(
    {"_id":{
        '_metadata': {
            "Date" : "날짜 (DatetimeIndex)",
            "Rank": "시가총액 순위 (당일)",
            "Code" : "종목코드",
            "Name" : "종명이름",
            "Open" : "시가",
            "High" : "고가",
            "Low" : "저가",
            "Close" : "종가",
            "Volume" : "거래량",
            "Amount" : "거래대금",
            "Changes" : "전일대비",
            "ChangeCode": "등락 기호",
            "ChagesRatio" : "전일대비 등락률",
            "Marcap" : "시가총액(백만원)",
            "Stocks" : "상장주식수",
            "MarketId" : "시장기호",
            "Market" : "시장",
            "Dept" : "부서(한국거래소)",
        }
    }
    }
)

InsertOneResult({'_metadata': {'Date': '날짜 (DatetimeIndex)', 'Rank': '시가총액 순위 (당일)', 'Code': '종목코드', 'Name': '종명이름', 'Open': '시가', 'High': '고가', 'Low': '저가', 'Close': '종가', 'Volume': '거래량', 'Amount': '거래대금', 'Changes': '전일대비', 'ChangeCode': '등락 기호', 'ChagesRatio': '전일대비 등락률', 'Marcap': '시가총액(백만원)', 'Stocks': '상장주식수', 'MarketId': '시장기호', 'Market': '시장', 'Dept': '부서(한국거래소)'}}, acknowledged=True)

In [5]:
collection.create_index([('Date', 1)])
collection.create_index([('Code', 1)])
collection.create_index([('Name', 1)])
collection.create_index([('Code', 1), ('Date', 1)])
collection.create_index([('Name', 1), ('Date', 1)])
collection.create_index([('filename', 1)])


'Name_1_Date_1'

In [19]:
import pandas as pd
today_df = pd.read_csv("https://github.com/FinanceData/marcap/raw/master/data/marcap-2024.csv.gz")
today_df['_id'] = today_df['Code'].astype(str)+"_"+today_df['Date']
today_df['filename'] = "marcap-2024.csv.gz"

In [20]:
query = { 'filename': 'value_to_search' }  
cursor = collection.find({"filename": "marcap-2024.csv.gz"})
mongo_df = pd.DataFrame(cursor)

In [28]:
insert_df = today_df[~today_df.isin(mongo_df)].dropna()

In [29]:
insert_df

,Code,Name,Close,Dept,ChangeCode,Changes,ChagesRatio,Volume,Amount,Open,High,Low,Marcap,Stocks,Market,MarketId,Rank,Date,_id,filename
12,247540,에코프로비엠,283500.0,우량기업부,2,-4500.0,-1.56,441580.0,1.261030e+11,288000.0,293000.0,282000.0,2.772668e+13,97801344,KOSDAQ GLOBAL,KSQ,13,2024-01-02,247540_2024-01-02,marcap-2024.csv.gz
19,086520,에코프로,638000.0,우량기업부,2,-9000.0,-1.39,249955.0,1.610081e+11,648000.0,660000.0,637000.0,1.698845e+13,26627668,KOSDAQ,KSQ,20,2024-01-02,086520_2024-01-02,marcap-2024.csv.gz
30,091990,셀트리온헬스케어,75900.0,우량기업부,0,0.0,0.00,0.0,0.000000e+00,0.0,0.0,0.0,1.248287e+13,164464724,KOSDAQ GLOBAL,KSQ,31,2024-01-02,091990_2024-01-02,marcap-2024.csv.gz
53,066970,엘앤에프,205500.0,우량기업부,1,1500.0,0.74,882774.0,1.846733e+11,204500.0,217000.0,200000.0,7.448928e+12,36247825,KOSDAQ GLOBAL,KSQ,54,2024-01-02,066970_2024-01-02,marcap-2024.csv.gz
58,028300,HLB,53200.0,중견기업부,1,2500.0,4.93,2937109.0,1.540048e+11,50900.0,54400.0,50400.0,6.878424e+12,129293688,KOSDAQ,KSQ,59,2024-01-02,028300_2024-01-02,marcap-2024.csv.gz
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201166,217320,썬테크,2500.0,일반기업부,0,0.0,0.00,0.0,0.000000e+00,0.0,0.0,0.0,2.625000e+09,1050000,KONEX,KNX,2799,2024-04-16,217320_2024-04-16,marcap-2024.csv.gz
201167,413300,티엘엔지니어링,1950.0,일반기업부,2,-50.0,-2.50,26.0,5.166000e+04,2280.0,2280.0,1950.0,2.545754e+09,1305515,KONEX,KNX,2800,2024-04-16,413300_2024-04-16,marcap-2024.csv.gz
201168,245450,씨앤에스링크,1599.0,일반기업부,1,99.0,6.60,11.0,1.659900e+04,1500.0,1599.0,1500.0,2.526356e+09,1579960,KONEX,KNX,2801,2024-04-16,245450_2024-04-16,marcap-2024.csv.gz
201169,308700,테크엔,199.0,일반기업부,0,0.0,0.00,0.0,0.000000e+00,0.0,0.0,0.0,7.960000e+08,4000000,KONEX,KNX,2802,2024-04-16,308700_2024-04-16,marcap-2024.csv.gz


In [4]:
all_documents = collection.find()
for document in all_documents:
    print(document)
    input()

{'_id': '005930_2015-01-02', 'Code': '005930', 'Name': '삼성전자', 'Market': 'KOSPI', 'Dept': nan, 'Close': 1330000.0, 'ChangeCode': 1, 'Changes': 3000.0, 'ChagesRatio': 0.23, 'Open': 1340000.0, 'High': 1340000.0, 'Low': 1327000.0, 'Volume': 175499.0, 'Amount': 230643147000.0, 'Marcap': 195908118210000.0, 'Stocks': 147299337, 'MarketId': 'STK', 'Rank': 1, 'Date': '2015-01-02', 'filename': 'marcap-2015.csv.gz'}
{'_id': '005380_2015-01-02', 'Code': '005380', 'Name': '현대차', 'Market': 'KOSPI', 'Dept': nan, 'Close': 169000.0, 'ChangeCode': 3, 'Changes': 0.0, 'ChagesRatio': 0.0, 'Open': 169000.0, 'High': 170500.0, 'Low': 167000.0, 'Volume': 458014.0, 'Amount': 77441258500.0, 'Marcap': 37226724951000.0, 'Stocks': 220276479, 'MarketId': 'STK', 'Rank': 2, 'Date': '2015-01-02', 'filename': 'marcap-2015.csv.gz'}


KeyboardInterrupt: Interrupted by user